In [1]:
from traceback import print_tb
import networkx as nx
import csv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
import pandas as pd

In [2]:
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(client_id="85aa8b30835e45a3b99623b2797df916",client_secret="ecb8aa6ad9d7431db74fb99af51dec93"))


  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/ab/63/966db1321a0ad55df1d1fe51505d2cdae191b84c907974873817b0a6e849/pandas-2.2.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for numpy>=1.23.2 from https://files.pythonhosted.org/packages/3f/6b/5610004206cf7f8e7ad91c5a85a8c71b2f2f8051a0c0c4d5916b76d6cbb2/numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata
  Using cached numpy-1.26.4-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2024.1-

ERROR: Could not install packages due to an OSError: [WinError 2] Impossibile trovare il file specificato: 'c:\\Python311\\Scripts\\f2py.exe' -> 'c:\\Python311\\Scripts\\f2py.exe.deleteme'


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
people = list()
with open('global.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        people.append(row)

dataset = list()
for person in people:
    
    playlist = spotify.playlist(person["Link"])
    songs = spotify.playlist_items(playlist["id"])
    #print(len(songs))
    aux = set()
    
    for i in range (len(songs["items"])):
        try: 
            genres = spotify.artist(songs["items"][i]["track"]["artists"][i]["id"])['genres']
            for genre in genres:
                aux.add(genre)
            dataset.append(aux)
        except:
            continue   
#print(dataset)

In [4]:
grafo = nx.Graph()

for genres in dataset:
    for genre1 in genres:
        for genre2 in genres:
            grafo.add_node(genre1)
            if genre1 != genre2 and not grafo.has_edge(genre2, genre1):
                grafo.add_edge(genre1, genre2)
nx.write_gexf(grafo, 'global_genres06.gexf')
with open('global_genres06.pkl', 'wb') as f:
    pickle.dump(grafo, f)